### Importing essential libraries

In [1]:
import pandas as pd
import mysql.connector as connector
import json
import numpy as np

### Importing Data

In [2]:
df = pd.read_excel('Global Super Store dataset.xlsx')
df.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium


### Preparing Data

Note that this is done along with Excel as well because it's easier to understand the sheet much better so that when writing scripts later to automate this process, it's easier

In [3]:
#Let's find missing values in each column
print('Missing Values in Each Column\n')
for i in df:
    print(f'{i} : {sum(df[i].isna())}')

Missing Values in Each Column

Row ID : 0
Order ID : 0
Order Date : 0
Ship Date : 0
Ship Mode : 0
Customer ID : 0
Customer Name : 0
Segment : 0
City : 0
State : 0
Country : 0
Postal Code : 41296
Market : 0
Region : 0
Product ID : 0
Category : 0
Sub-Category : 0
Product Name : 0
Sales : 0
Quantity : 0
Discount : 0
Profit : 0
Shipping Cost : 0
Order Priority : 0


In [3]:
#Let's add a new calculated field as "Unit Price"
df['Unit Price']= (df['Sales'] / (1-df['Discount'])) / df['Quantity']
df.tail(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority,Unit Price
51288,9596,MX-2012-140767,18-02-2012,22-02-2012,Standard Class,RB-19795,Ross Baird,Home Office,Valinhos,São Paulo,...,Office Supplies,Binders,"Acco Index Tab, Economy",13.44,2,0.0,2.4,0.0,Medium,6.72
51289,6147,MX-2012-134460,22-05-2012,26-05-2012,Second Class,MC-18100,Mick Crebagga,Consumer,Tipitapa,Managua,...,Office Supplies,Paper,"Eaton Computer Printout Paper, 8.5 x 11",61.38,3,0.0,1.8,0.0,High,20.46


### Extracting Table Entities from the Original DF

**Here's our ER Diagram for the table**

<div><img src="ERD.png" width="500"/></div>

In [4]:
#Let's create dataframes that represents each table

customers_cols = ['Customer ID', 'Customer Name', 'Segment']
locations_cols = ['City', 'State', 'Country', 'Postal Code', 'Market', 'Region']
orders_cols = ['Order ID', 'Order Date', 'Sales', 'Quantity', 'Discount', 'Profit', 'Order Priority', 'Ship Date', 'Ship Mode', 'Shipping Cost', 'Customer ID']
products_cols = ['Product ID', 'Category', 'Sub-Category', 'Product Name', 'Unit Price']

In [5]:
table_names = ['Customers', 'Locations', 'Orders', 'Products']
vals = [customers_cols, locations_cols, orders_cols, products_cols]

In [6]:
tbs = {} #To store each table object temporarily before dropping duplicates

for i in range(4):
    temp_tb = df[vals[i]]
    tbs[table_names[i]] = temp_tb

In [7]:
tbs['Locations'].head(3)

,City,State,Country,Postal Code,Market,Region
0,New York City,New York,United States,10024.0,US,East
1,Wollongong,New South Wales,Australia,NaN,APAC,Oceania
2,Brisbane,Queensland,Australia,NaN,APAC,Oceania


In [8]:
for key,val in tbs.items():
    print(f'Length of <{key}> table: {len(val)}')

Length of <Customers> table: 51290
Length of <Locations> table: 51290
Length of <Orders> table: 51290
Length of <Products> table: 51290


### Let's get rid of the duplicates in each table

In [9]:
unique_tables = {} #To store new tables

#Let's remove duplicates for customers, orders, and products tables and add to unique tables disctionary (because they are straightforward)
unique_tables['Customer_tab'] = tbs['Customers'].drop_duplicates()
unique_tables['Orders_tab'] = tbs['Orders'].drop_duplicates()
unique_tables['Products_tab'] = tbs['Products'].drop_duplicates()

In [10]:
#Since Locations need a primary key, lets remove duplicates, add a primary key for unique values and store it in the dictionary object
loc_temp = tbs['Locations'].drop_duplicates()
#Adding the Location ID column
loc_temp.insert(0, 'Location ID', range(1, len(loc_temp) + 1))
unique_tables['locations_tab'] = loc_temp

In [11]:
for key, val in unique_tables.items():
    print(f'Lenth of <{key}> table : {len(val)}')

Lenth of <Customer_tab> table : 1590
Lenth of <Orders_tab> table : 51290
Lenth of <Products_tab> table : 17240
Lenth of <locations_tab> table : 3847


Orders is the same size as master table. To make this table unique we need to group by each order together and add it's sales, profit and ship cost together.

Right now all the main tables are done. Now the challenging part. Creating the `Products-Orders` table & `Locations-Customers` table

Let's make the `Products-Orders` table. Here since some products share the same code, we are going to use the `Unit Price` of that product. This is why we added the `Unit Price` caculated field in the beginning of the code.

For example let's say Product ID is 004567. It has 2 products as "Leather Bat", "Mud guard". When this happens we can't distinguish between which product the customer has ordered when joining the orders table and products table for detailed view. Because of that we are using Unit price. So if the "Leather Bat" costs `$4.8` & "Mud guard" costs `$0.8`. We know what product actually the customer ordered for the Order's product ID of 004567 by calculating it's unit price.

In [12]:
#Let's create the orders-products table
orders_products_temp = df[['Order ID', 'Product ID', 'Unit Price', 'Quantity', 'Discount', 'Profit', 'Shipping Cost']]
orders_products = orders_products_temp.drop_duplicates()
orders_products.tail(6)

,Order ID,Product ID,Unit Price,Quantity,Discount,Profit,Shipping Cost
51284,IN-2014-57662,OFF-BI-10002424,12.90,5,0.1,19.9500,0.01
51285,IN-2014-62366,OFF-FA-10000746,13.02,5,0.0,4.5000,0.01
51286,US-2014-102288,OFF-AP-10002906,2.22,1,0.8,-1.1100,0.01
51287,US-2013-155768,OFF-EN-10001219,7.64,3,0.0,11.2308,0.01
51288,MX-2012-140767,OFF-BI-10000806,6.72,2,0.0,2.4000,0.00
51289,MX-2012-134460,OFF-PA-10004155,20.46,3,0.0,1.8000,0.00


Now Let's create the `Locations-Customers` table. For this we need to create two temporary tables. 

* First we need a temporary location table replicate with an additional concat string to match along
* Second we need another temporary table for location table with customer ids without duplicates with the same concat string
* Then we need to check the location ID and add it to the second temporary table using Concat String
* Then we need to extract only the Location ID & Customer ID from the second temporary table & remove it's duplicates

In [13]:
#First table is the replicate of locations table with Concat String
temp_1 = unique_tables['locations_tab'].copy()
temp_1['Postal Code'] = temp_1['Postal Code'].astype(str) #Convert postal code to a string or an error will occur when adding concat string
temp_1['Concat String'] = temp_1[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)

In [14]:
#Notice we made a copy of locations table to produce the first temporary table needed to make the locations-customers table
#Check the differences in the columns
print(temp_1.columns)
print(unique_tables['locations_tab'].columns)

Index(['Location ID', 'City', 'State', 'Country', 'Postal Code', 'Market',
       'Region', 'Concat String'],
      dtype='object')
Index(['Location ID', 'City', 'State', 'Country', 'Postal Code', 'Market',
       'Region'],
      dtype='object')


In [15]:
temp_1.head(4)

,Location ID,City,State,Country,Postal Code,Market,Region,Concat String
0,1,New York City,New York,United States,10024.0,US,East,New York City-New York-United States-US-East-1...
1,2,Wollongong,New South Wales,Australia,nan,APAC,Oceania,Wollongong-New South Wales-Australia-APAC-Ocea...
2,3,Brisbane,Queensland,Australia,nan,APAC,Oceania,Brisbane-Queensland-Australia-APAC-Oceania-nan
3,4,Berlin,Berlin,Germany,nan,EU,Central,Berlin-Berlin-Germany-EU-Central-nan


In [16]:
#Next is the location table with customer ids
temp_2_temp = df[['City', 'State', 'Country', 'Postal Code', 'Market', 'Region', 'Customer ID']]
print(f'Before removing duplicates from second table: {len(temp_2_temp)}')
temp_2 = temp_2_temp.drop_duplicates() #When dropping duplicates, it needs to be saved to a new variable. Otherwise effects won't commit
print(f'After removing duplicates from second table: {len(temp_2)}')

Before removing duplicates from second table: 51290
After removing duplicates from second table: 25181


In [17]:
temp_2['Postal Code'] = temp_2['Postal Code'].astype(str)
temp_2['Concat String'] = temp_2[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)
temp_2.head(3)

C:\Users\USER\AppData\Local\Temp\ipykernel_4092\1592578394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_2['Postal Code'] = temp_2['Postal Code'].astype(str)
C:\Users\USER\AppData\Local\Temp\ipykernel_4092\1592578394.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_2['Concat String'] = temp_2[['City', 'State', 'Country', 'Market', 'Region', 'Postal Code']].agg('-'.join, axis=1)


,City,State,Country,Postal Code,Market,Region,Customer ID,Concat String
0,New York City,New York,United States,10024.0,US,East,RH-19495,New York City-New York-United States-US-East-1...
1,Wollongong,New South Wales,Australia,nan,APAC,Oceania,JR-16210,Wollongong-New South Wales-Australia-APAC-Ocea...
2,Brisbane,Queensland,Australia,nan,APAC,Oceania,CR-12730,Brisbane-Queensland-Australia-APAC-Oceania-nan


Pandas Documentation About Merge: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html

In [18]:
#Let's match everything
result = pd.merge(temp_2, temp_1, on='Concat String', how='left')
result.head(3)

,City_x,State_x,Country_x,Postal Code_x,Market_x,Region_x,Customer ID,Concat String,Location ID,City_y,State_y,Country_y,Postal Code_y,Market_y,Region_y
0,New York City,New York,United States,10024.0,US,East,RH-19495,New York City-New York-United States-US-East-1...,1,New York City,New York,United States,10024.0,US,East
1,Wollongong,New South Wales,Australia,nan,APAC,Oceania,JR-16210,Wollongong-New South Wales-Australia-APAC-Ocea...,2,Wollongong,New South Wales,Australia,nan,APAC,Oceania
2,Brisbane,Queensland,Australia,nan,APAC,Oceania,CR-12730,Brisbane-Queensland-Australia-APAC-Oceania-nan,3,Brisbane,Queensland,Australia,nan,APAC,Oceania


In [19]:
#Let's extract only the required: Location ID & Customer ID
result_temp = result[['Customer ID','Location ID']]
customers_locations = result_temp.drop_duplicates()
(f'Length of <Customer-Locations> Table: {len(customers_locations)}')

'Length of <Customer-Locations> Table: 25181'

In [20]:
customers_locations.head(3)

,Customer ID,Location ID
0,RH-19495,1
1,JR-16210,2
2,CR-12730,3


In [22]:
unique_tables['orders_products'][unique_tables['orders_products']['Order ID'] == 'CA-2012-124891']

,Order ID,Product ID,Unit Price,Quantity,Discount,Profit,Shipping Cost
0,CA-2012-124891,TEC-AC-10003033,329.95,7,0.0,762.1845,933.57
331,CA-2012-124891,FUR-TA-10002903,259.71,7,0.4,-290.8752,348.18
26575,CA-2012-124891,OFF-PA-10004621,6.48,3,0.0,9.3312,7.24


In [23]:
unique_tables['Orders_tab'][unique_tables['Orders_tab']['Order ID'] == 'CA-2012-124891']

,Order ID,Order Date,Sales,Quantity,Discount,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,CA-2012-124891,31-07-2012,2309.650,7,0.0,762.1845,Critical,31-07-2012,Same Day,933.57,RH-19495
331,CA-2012-124891,31-07-2012,1090.782,7,0.4,-290.8752,Critical,31-07-2012,Same Day,348.18,RH-19495
26575,CA-2012-124891,31-07-2012,19.440,3,0.0,9.3312,Critical,31-07-2012,Same Day,7.24,RH-19495


In [21]:
#Adding the Customer-Locations & Product-Order tables into "unique_tables" dictionary
unique_tables['customers_locations'] = customers_locations
unique_tables['orders_products'] = orders_products

Now let's make the changes in the `Orders` table.

* First we need to drop the unnecessary columns which are, "Quantity" & "Discount". We don't need them because they are in the `orders-products` table
* Then we are going to group by "Order ID", add "Sales", "Profit" & "Shipping Cost" columns together and eleminate the repeating groups

In [24]:
#Dropping "Quantity" & "Discount" from orders table
unique_tables['Orders_tab'].drop(["Quantity", "Discount"], axis=1, inplace=True)
unique_tables['Orders_tab'].head(2)

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,CA-2012-124891,31-07-2012,2309.650,762.1845,Critical,31-07-2012,Same Day,933.57,RH-19495
1,IN-2013-77878,05-02-2013,3709.395,-288.7650,Critical,07-02-2013,Second Class,923.63,JR-16210


In [26]:
#Creating a copy just in case
orders = unique_tables['Orders_tab'].copy()
orders.head(2)

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,CA-2012-124891,31-07-2012,2309.650,762.1845,Critical,31-07-2012,Same Day,933.57,RH-19495
1,IN-2013-77878,05-02-2013,3709.395,-288.7650,Critical,07-02-2013,Second Class,923.63,JR-16210


In [28]:
agg_orders = orders.groupby('Order ID').agg({
    'Order Date' : 'first',
    'Sales' : 'sum',
    'Profit' : 'sum',
    'Order Priority' : 'first',
    'Ship Date' : 'first',
    'Ship Mode' : 'first',
    'Shipping Cost' : 'sum',
    'Customer ID' : 'first'    
}).reset_index()

In [29]:
agg_orders.head(4)

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,AE-2011-9160,03-10-2011,161.082,-246.078,Medium,07-10-2011,Standard Class,9.56,PO-8865
1,AE-2013-1130,14-10-2013,228.996,-236.964,High,14-10-2013,Same Day,60.18,EB-4110
2,AE-2013-1530,31-12-2013,23.634,-38.076,High,03-01-2014,Second Class,3.16,MY-7380
3,AE-2014-2840,05-11-2014,42.480,-75.060,Critical,08-11-2014,First Class,8.04,PG-8820


len(agg_orders)

In [31]:
len(unique_tables['Orders_tab'])

51290

In [32]:
#Let's replace orders table's value with unique order_id table we got in the previous step
unique_tables['Orders_tab'] = agg_orders

In [33]:
len(unique_tables['Orders_tab'])

25035

When opened with Excel, I found that there are duplicates for the `Products` table:

Therefore, we need to remove duplicates specific column wise.

For products table, it's product_id which causes the problem (refer to the code below to get an idea from 1 instance)

In [34]:
#There are duplicate entries for the following product id in Products table, even after removing duplicates
unique_tables['Products_tab'][unique_tables['Products_tab']['Product ID'] == 'TEC-AC-10003033'] 

,Product ID,Category,Sub-Category,Product Name,Unit Price
0,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,329.95
3021,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,329.95


In [35]:
#Let's remove the duplicates
print(f'Before removing duplicates: {len(unique_tables['Products_tab'])}')
unique_tables['Products_tab'].drop_duplicates('Product ID', inplace=True)
print(f'After removing duplicates: {len(unique_tables['Products_tab'])}')

Before removing duplicates: 17240
After removing duplicates: 10292


C:\Users\USER\AppData\Local\Temp\ipykernel_4092\472613525.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_tables['Products_tab'].drop_duplicates('Product ID', inplace=True)


In [36]:
#Let's check final number of rows in each table
for key, value in unique_tables.items():
    print(f'Length of table <{key}> : {len(value)}')

Length of table <Customer_tab> : 1590
Length of table <Orders_tab> : 25035
Length of table <Products_tab> : 10292
Length of table <locations_tab> : 3847
Length of table <customers_locations> : 25181
Length of table <orders_products> : 51290


### Exporting all tables

In [37]:
#Download every table in excel formats if needed
unique_tables['Customer_tab'].to_excel('customers.xlsx', index=False)
unique_tables['Orders_tab'].to_excel('orders.xlsx', index=False)
unique_tables['Products_tab'].to_excel('products.xlsx', index=False)
unique_tables['locations_tab'].to_excel('locations.xlsx', index=False)
unique_tables['customers_locations'].to_excel('customer_locations.xlsx', index=False)
unique_tables['orders_products'].to_excel('order_products.xlsx', index=False)

### Import tables created previously

In case if you have the tables exported from the previous code, and you want to engage in the acitivty of playing with SQL with those data tables we just created, we can use the following functions to get those tables and store them as data frames without having to re-run the code from the beginning except importing libraries

In [2]:
tables = {} #An empty dictionary to store tables

def getTables():
    customers = pd.read_excel('REVAMPED TABLES/customers.xlsx')
    locations = pd.read_excel('REVAMPED TABLES/locations.xlsx')
    orders = pd.read_excel('REVAMPED TABLES/orders.xlsx')
    products = pd.read_excel('REVAMPED TABLES/products.xlsx')
    customer_location = pd.read_excel('REVAMPED TABLES/customer_locations.xlsx')
    orders_products = pd.read_excel('REVAMPED TABLES/order_products.xlsx')
    return [customers, locations, orders, products, customer_location, orders_products]

table_holder = getTables()

tables['Customers']= table_holder[0]
tables['Locations']= table_holder[1]
tables['Orders']= table_holder[2]
tables['Products']= table_holder[3]
tables['customer_locations']= table_holder[-2]
tables['orders_products']= table_holder[-1]

In [3]:
tables['Products'].head()

,Product ID,Category,Sub-Category,Product Name,Unit Price
0,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,329.95
1,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",457.95
2,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",638.91
3,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",642.78
4,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",354.12


In [4]:
for key, val in tables.items():
    print(f'Length of <{key}> table : {len(val)}')

Length of <Customers> table : 1590
Length of <Locations> table : 3847
Length of <Orders> table : 25035
Length of <Products> table : 10292
Length of <customer_locations> table : 25181
Length of <orders_products> table : 51290


### Creating the Database

**Here's our ERD for the database**

<div><img src="ERD (Proper).png" width="500"/></div>

In [5]:
#Extracting DB credentials
with open('db_credentials.json', 'r') as f:
    db_creds = json.load(f)

In [6]:
#Let's establish a connection between MySQL & Python
conn = connector.connect(user = db_creds['user'], password = db_creds['password'])
cur = conn.cursor()

In [7]:
#A function to display outputs as Pandas DFs
def runQ(query):
    cur.execute(query)
    rows = cur.fetchall()
    cols = cur.column_names
    return pd.DataFrame(rows, columns = cols)

In [8]:
runQ('SHOW DATABASES')

,Database
0,information_schema
1,little_lemon
2,lucky_shrub
3,m&g_schema
4,mangata_gallo
5,mysql
6,performance_schema
7,sys
8,temporary
9,university


In [9]:
#Queries to create the new database
cur.execute('CREATE DATABASE global_super_store;')
cur.execute('USE global_super_store;') #To use the database needed
conn.database #Check if we are using the correct database

'global_super_store'

In [10]:
#Let's see if the database is in the MySQL database list
runQ('SHOW DATABASES;')

,Database
0,global_super_store
1,information_schema
2,little_lemon
3,lucky_shrub
4,m&g_schema
5,mangata_gallo
6,mysql
7,performance_schema
8,sys
9,temporary


In [11]:
#Queries to create tables
customers = '''CREATE TABLE Customers (
CustomerID VARCHAR(20) PRIMARY KEY NOT NULL, CustomerName VARCHAR(50) NOT NULL, Segment VARCHAR(20) NOT NULL
);'''

orders = '''CREATE TABLE Orders (
OrderID VARCHAR(25) PRIMARY KEY NOT NULL, OrderDate DATE NOT NULL, Sales DECIMAL(8,3) NOT NULL, Profits DECIMAL(7,3) NOT NULL, 
OrderPriority VARCHAR(20) NOT NULL, ShipDate DATE NOT NULL, ShipMode VARCHAR(25) NOT NULL, ShippingCost DECIMAL(7,3) NOT NULL, 
CustomerID VARCHAR(20) NOT NULL, FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID) ON DELETE CASCADE ON UPDATE CASCADE
);'''

locations = '''CREATE TABLE Locations (
LocationID INT PRIMARY KEY NOT NULL, City VARCHAR(45) NOT NULL, State VARCHAR(45) NOT NULL, Country VARCHAR(45) NOT NULL, PostalCode INT,
Market VARCHAR(15) NOT NULL, Region VARCHAR(25) NOT NULL
);'''

products = '''CREATE TABLE Products (
ProductID VARCHAR(25) PRIMARY KEY NOT NULL, Category VARCHAR(30) NOT NULL, Subcategory VARCHAR(30) NOT NULL, ProductName VARCHAR(175) NOT NULL,
UnitPrice DECIMAL(7,3) NOT NULL
);'''

customer_locations = '''CREATE TABLE Customer_Locations (
CustomerID VARCHAR(20) NOT NULL, LocationID INT NOT NULL, PRIMARY KEY (CustomerID, LocationID), 
FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID) ON UPDATE CASCADE ON DELETE CASCADE,
FOREIGN KEY (LocationID) REFERENCES Locations(LocationID) ON UPDATE CASCADE ON DELETE CASCADE
);'''

orders_products = '''CREATE TABLE Orders_Products (
OrderID VARCHAR(25) NOT NULL, ProductID VARCHAR(25) NOT NULL, UnitPrice DECIMAL(7,3) NOT NULL, Quantity INT NOT NULL, Discount DECIMAL(3,2) NOT NULL,
Profits DECIMAL(7,3) NOT NULL, ShippingCost DECIMAL(7,3) NOT NULL, PRIMARY KEY (OrderID, ProductID), 
FOREIGN KEY (OrderID) REFERENCES Orders(OrderID) ON UPDATE CASCADE ON DELETE CASCADE,
FOREIGN KEY (ProductID) REFERENCES Products(ProductID) ON UPDATE CASCADE ON DELETE CASCADE
);'''

In [12]:
table_queries = [customers, orders, locations, products, customer_locations, orders_products]

In [13]:
for query in table_queries:
    cur.execute(query)

In [14]:
runQ('SHOW TABLES;')

,Tables_in_global_super_store
0,customer_locations
1,customers
2,locations
3,orders
4,orders_products
5,products


In [15]:
#To check if the columns are created correctly
runQ('SHOW COLUMNS FROM Locations;')

,Field,Type,Null,Key,Default,Extra
0,LocationID,int,NO,PRI,None,
1,City,varchar(45),NO,,None,
2,State,varchar(45),NO,,None,
3,Country,varchar(45),NO,,None,
4,PostalCode,int,YES,,None,
5,Market,varchar(15),NO,,None,
6,Region,varchar(25),NO,,None,


### Populating Tables

In [16]:
#Queries to populate each table
pop_customers = 'INSERT INTO Customers(CustomerID, CustomerName, Segment) VALUES (%s, %s, %s);'

pop_orders = '''INSERT INTO Orders(OrderID, OrderDate, Sales, Profits, OrderPriority, ShipDate, ShipMode, ShippingCost, CustomerID) 
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);'''

pop_locations = '''INSERT INTO Locations(LocationID, City, State, Country, PostalCode, Market, Region) VALUES (%s, %s, %s, %s, %s, %s, %s);'''

pop_products = '''INSERT INTO Products(ProductID, Category, Subcategory, ProductName, UnitPrice) VALUES (%s, %s, %s, %s, %s);'''

pop_order_products = '''INSERT INTO Orders_Products(OrderID, ProductID, UnitPrice, Quantity, Discount, Profits, ShippingCost) 
VALUES(%s, %s, %s, %s, %s, %s, %s);'''

pop_customer_locations = '''INSERT INTO Customer_Locations(CustomerID, LocationID) VALUES (%s, %s);'''

Order of the `pop_queries` list is important when looping to populate

In [17]:
pop_queries = [pop_customers, pop_locations, pop_orders, pop_products, pop_customer_locations, pop_order_products]

### Problem 01: Converting NaN data types to None for Parsing to SQL

I identified that when location table getting populated, it raises an error when PostalCode column contains missing values. In Pandas they are known as NaN values. But in SQL it's NULL. So we need to convert NaN values to None before we insert them into SQL

In [18]:
location_temp = tables['Locations'].copy()

In [19]:
location_temp = location_temp.replace(np.nan, None)

Here, see the difference between None type values and NaN type values. SQL can only read None from Python as NULL when populating data

In [21]:
location_temp.head(4)

,Location ID,City,State,Country,Postal Code,Market,Region
0,1,New York City,New York,United States,10024.0,US,East
1,2,Wollongong,New South Wales,Australia,None,APAC,Oceania
2,3,Brisbane,Queensland,Australia,None,APAC,Oceania
3,4,Berlin,Berlin,Germany,None,EU,Central


In [22]:
#See missing values from Postal Code is shown as NaN in the original table
tables['Locations'].head()

,Location ID,City,State,Country,Postal Code,Market,Region
0,1,New York City,New York,United States,10024.0,US,East
1,2,Wollongong,New South Wales,Australia,NaN,APAC,Oceania
2,3,Brisbane,Queensland,Australia,NaN,APAC,Oceania
3,4,Berlin,Berlin,Germany,NaN,EU,Central
4,5,Dakar,Dakar,Senegal,NaN,Africa,Africa


In [23]:
tables['Locations'] = location_temp

In [24]:
tables['Locations'].head()

,Location ID,City,State,Country,Postal Code,Market,Region
0,1,New York City,New York,United States,10024.0,US,East
1,2,Wollongong,New South Wales,Australia,None,APAC,Oceania
2,3,Brisbane,Queensland,Australia,None,APAC,Oceania
3,4,Berlin,Berlin,Germany,None,EU,Central
4,5,Dakar,Dakar,Senegal,None,Africa,Africa


### Problem 02: Converting Date formats to MySQL format

I found that it throws an error when trying to populate `orders` table regarding date column format. The given format in Pandas DFs is not the format we need for the SQL parsing. Therefore we need to convert Date Fromat as well.

In [25]:
from datetime import datetime as dt

In [26]:
#Create a function needed to convert the date format
def convertDate(date_obj):
    return dt.strptime(date_obj, '%d-%m-%Y').strftime('%Y-%m-%d')    

In [27]:
temp_orders = tables['Orders'].copy()

temp_orders['Order Date'] = temp_orders['Order Date'].apply(convertDate)
temp_orders['Ship Date'] = temp_orders['Ship Date'].apply(convertDate)

In [28]:
temp_orders.head()

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,AE-2011-9160,2011-10-03,161.082,-246.078,Medium,2011-10-07,Standard Class,9.56,PO-8865
1,AE-2013-1130,2013-10-14,228.996,-236.964,High,2013-10-14,Same Day,60.18,EB-4110
2,AE-2013-1530,2013-12-31,23.634,-38.076,High,2014-01-03,Second Class,3.16,MY-7380
3,AE-2014-2840,2014-11-05,42.480,-75.060,Critical,2014-11-08,First Class,8.04,PG-8820
4,AE-2014-3830,2014-12-13,281.502,-429.108,Medium,2014-12-19,Standard Class,19.38,GH-4665


In [29]:
tables['Orders'].head(3) #This is how the date is in the original table

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,AE-2011-9160,03-10-2011,161.082,-246.078,Medium,07-10-2011,Standard Class,9.56,PO-8865
1,AE-2013-1130,14-10-2013,228.996,-236.964,High,14-10-2013,Same Day,60.18,EB-4110
2,AE-2013-1530,31-12-2013,23.634,-38.076,High,03-01-2014,Second Class,3.16,MY-7380


In [30]:
#Let's replace original table
tables['Orders'] = temp_orders
tables['Orders'].head()

,Order ID,Order Date,Sales,Profit,Order Priority,Ship Date,Ship Mode,Shipping Cost,Customer ID
0,AE-2011-9160,2011-10-03,161.082,-246.078,Medium,2011-10-07,Standard Class,9.56,PO-8865
1,AE-2013-1130,2013-10-14,228.996,-236.964,High,2013-10-14,Same Day,60.18,EB-4110
2,AE-2013-1530,2013-12-31,23.634,-38.076,High,2014-01-03,Second Class,3.16,MY-7380
3,AE-2014-2840,2014-11-05,42.480,-75.060,Critical,2014-11-08,First Class,8.04,PG-8820
4,AE-2014-3830,2014-12-13,281.502,-429.108,Medium,2014-12-19,Standard Class,19.38,GH-4665


### Problem 03: Duplicate primary keys in Orders_Products

Need to eliminate them. Best example found was `Order ID` = CA-2014-152912 & `Product ID` = OFF-ST-10003208

In [31]:
temp_orders_products = tables['orders_products'].copy()
temp_orders_products[(temp_orders_products['Order ID'] == 'CA-2014-152912') & (temp_orders_products['Product ID'] == 'OFF-ST-10003208')]

,Order ID,Product ID,Unit Price,Quantity,Discount,Profit,Shipping Cost
778,CA-2014-152912,OFF-ST-10003208,181.46,9,0.0,473.6106,232.86
1061,CA-2014-152912,OFF-ST-10003208,181.46,3,0.0,157.8702,196.07


In [32]:
temp_orders_products.drop_duplicates(['Order ID', 'Product ID'], inplace=True) #Removing duplicates
temp_orders_products[(temp_orders_products['Order ID'] == 'CA-2014-152912') & (temp_orders_products['Product ID'] == 'OFF-ST-10003208')] 

,Order ID,Product ID,Unit Price,Quantity,Discount,Profit,Shipping Cost
778,CA-2014-152912,OFF-ST-10003208,181.46,9,0.0,473.6106,232.86


In [33]:
#Number of Rows Before and After removing duplicates
len(tables['orders_products']), len(temp_orders_products)

(51290, 51252)

In [34]:
#Replacing with original table
tables['orders_products'] = temp_orders_products
len(tables['orders_products'])

51252

When inserting data into each table, we need to extract each row as a tuple. To do this we can iterate over each table's row, cconvert it into a tuple, then store all the tuples of one single table into a list. Then all the 6 lists will be added to one single **Master** list known as `data` to make it easy when populating tables through a looping function. 

In [35]:
#Let's create lists for each table where each record is stored as a tuple
data = []

for key, val in tables.items():
    temp = [tuple(row) for index, row in tables[key].iterrows()]
    data.append(temp)

In [36]:
#Let's try populate tables
for i in range(0,len(pop_queries)):
    cur.executemany(pop_queries[i], data[i])

In [43]:
#Checking if the tables are populated or not
runQ('SELECT * FROM Customer_Locations LIMIT 10;')

,CustomerID,LocationID
0,AH-10195,1
1,AH-10210,1
2,AH-10690,1
3,AJ-10795,1
4,AR-10510,1
5,AR-10540,1
6,AS-10225,1
7,AT-10735,1
8,AZ-10750,1
9,BD-11770,1


In [44]:
#Let's save the changes to the database tables 
conn.commit()

In [45]:
cur.close()
conn.close()